In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from tqdm.auto import tqdm
from multiprocessing import Pool
from recombinator.iid_bootstrap import iid_bootstrap
import recombinator.block_bootstrap as bb
from numba import jit
from functools import partial

sns.set_style('whitegrid')
sns.set_context("paper", font_scale=1.7)

# Main

## Get return data

In [7]:
returns_df = pd.read_json('https://retirementplans.vanguard.com/web/angular/app/nesteggcalculator/data/config.json')
returns_df = returns_df['data'].apply(pd.Series)
returns_df['year'] = returns_df['year'].astype(int)

returns_df.head()

,year,stocks,bonds,cash,cpi
0,1926,0.1163,0.0738,0.032,-0.0112
1,1927,0.3744,0.0743,0.031,-0.0226
2,1928,0.4360,0.0284,0.040,-0.0116
3,1929,-0.0855,0.0328,0.044,0.0058
4,1930,-0.2478,0.0796,0.022,-0.0640


## Simulation

In [13]:
class PortfolioSim:
    def __init__(self, returns_df, real_salary_yronyr_growth,
        real_avg_salary_yronyr_growth, base_yrly_salary):
        """
        returns_df:           the dataframe of returns from the VG site
        """
        
        self.returns_df = returns_df
        self.real_salary_mnthonmnth_growth = real_salary_yronyr_growth ** (1 / 12)
        self.real_avg_salary_mnthonmnth_growth = real_avg_salary_yronyr_growth ** (1 / 12)
        self.lower_taxband = 27295 / 12
        self.upper_taxband = 49130/12
        self.first_payment = "2021-10-20" # Day of first payment from student loan company to me, also day interest starts accruing
        self.last_repayment = "2056-04-05"
        start_date = pd.Timestamp(self.first_payment)
        end_date = pd.Timestamp(self.last_repayment)
        self.no_of_months = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month)
        self.base_yrly_salary = base_yrly_salary

    def gen_bootstrap_samples(self, bootstrap_replications, kind, block_length=5):
        # Generate bootstrap samples

        if kind == "iid":
            bootstrap_samples = iid_bootstrap(
                np.array(self.returns_df),
                replications=bootstrap_replications,
                sub_sample_length=self.no_of_months,
            )
        elif kind == "stationary":
            bootstrap_samples = bb.stationary_bootstrap(
                np.array(self.returns_df),
                block_length=block_length,
                replications=bootstrap_replications,
                sub_sample_length=self.no_of_months,
            )
        else: 
            raise Exception(f'Unknown bootstrap type: {kind}')

        self.bootstrap_samples = bootstrap_samples

    @staticmethod
    @jit(nopython=True)
    def sim_portfolio(bootstrap_sample, first_payment, lower_taxband, base_yrly_salary, real_salary_mnthonmnth_growth, real_avg_salary_mnthonmnth_growth, last_repayment):
        def monthly_income(base_yrly_salary, nominal_salary_index):
            return base_yrly_salary / 12 * nominal_salary_index # Need to adjust so that salary adjusts yearly.

        def taxband_monthly_income(nominal_avg_salary_index, monthly_band):
            return monthly_band * nominal_avg_salary_index # Need to adjust so that salary adjusts yearly.

        def monthly_repayment(nominal_avg_salary_index, lower_taxband, base_yrly_salary, nominal_salary_index):
            delta_income = monthly_income(base_yrly_salary, nominal_salary_index) - taxband_monthly_income(nominal_avg_salary_index, lower_taxband)
            if delta_income > 0:
                return delta_income * 0.09
            else:
                return 0

        def working_interest(nominal_avg_salary_index, lower_taxband, base_yrly_salary, nominal_salary_index):
            rpi = 1.0514448923219213 # fix this u nonce

            income = monthly_income(base_yrly_salary, nominal_salary_index)
            bottom_band = taxband_monthly_income(nominal_avg_salary_index,lower_taxband)
            upper_band = taxband_monthly_income(nominal_avg_salary_index,lower_taxband)

            delta_income = income - bottom_band
            if delta_income > 0:
                if income > upper_band:
                    return (rpi + 0.03) ** (1 / 365) # daily interest rate from annual interest rate
                else:
                    return (rpi + delta_income / (upper_band - bottom_band) * 0.03) ** (1 / 365)  # linear interpolation with 0% at 27295 and 3% at 49130
            else:
                    return 1
        payments = 0
        npv_payments = 0
        npv_SP500_payments = 0

        i = 0 # which month bootstrap we are on
        nominal_salary_index = 1
        nominal_avg_salary_index = 1

        # only used for NPV calculation, starts when loan is first paid out unlike the previous indices which start on first job
        cpih_index = 1 
        sp500_index = 1

        # Whilst in University
        daterange = pd.date_range(first_payment, "2026-04-05") # check the end date

        debt = 0
        rpi = 1.0514448923219213 # fix this u nonce !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        interest = (rpi + 0.03) ** (1 / 365) 

        for single_date in daterange:
            if pd.Timestamp(first_payment) == single_date or pd.Timestamp("2022-02-02") == single_date or pd.Timestamp("2022-10-19") == single_date or pd.Timestamp("2023-02-01") == single_date or pd.Timestamp("2023-10-18") == single_date or pd.Timestamp("2024-01-31") == single_date or pd.Timestamp("2024-10-17") == single_date or pd.Timestamp("2025-01-30") == single_date:
                debt += 4422/3 + .25 * 9250
            if pd.Timestamp("2022-05-04") == single_date or pd.Timestamp("2023-05-03") == single_date or pd.Timestamp("2024-05-02") == single_date or pd.Timestamp("2025-05-01") == single_date:
                debt += 4422/3 + .5 * 9250
            debt = debt * interest

            if single_date.day == 1:
                cpih_index *= bootstrap_sample[i, 1]
                sp500_index *= bootstrap_sample[i, 0]
                i += 1

        # After leaving University
        daterange = pd.date_range("2026-04-06", last_repayment)
        for single_date in daterange: 
            if single_date.day == 1:
                nominal_salary_index *= real_salary_mnthonmnth_growth * bootstrap_sample[i, 1]
                nominal_avg_salary_index *= real_avg_salary_mnthonmnth_growth * bootstrap_sample[i, 1] 
                # Real = Nominal / Price
                # Nominal (t + 1) = Real (t + 1) * Price (t + 1)
                # real_salary_mnthonmnth_growth = Real(t + 1) / Real (t). Real(t + 1) - Real (t) / Real(t) =  Real(t + 1) / Real(t) - 1
                # Nominal (t + 1) = Nominal(t) * Real(t + 1) / Real (t) * Price(t + 1) / Price (t)

                repayment = monthly_repayment(nominal_avg_salary_index, lower_taxband, base_yrly_salary, nominal_salary_index)
                i += 1
            else:
                repayment = 0

            interest = working_interest(nominal_avg_salary_index, lower_taxband, base_yrly_salary, nominal_salary_index) # interest is accrued daily whilst repayments are monthly so can't be inside previous if

            repayment = min(repayment, debt * interest) # if repayment > debt * interest
            debt = debt * interest - repayment

            payments += repayment

            npv_payments += repayment / cpih_index
            npv_SP500_payments += repayment / sp500_index

            if debt <= 0:                
                #print(single_date)
                break

        #print("payments: %s, NPV: %s, NPV SP500: %s" %(payments, npv_payments, npv_SP500_payments))
        #print(".", end = '')    
        return [payments, npv_payments, npv_SP500_payments]

    def run_simulation(self):

        # Output list
        yearly_balance_list = []

        # Partial function for parallel map
        run_trial = partial(
            self.sim_portfolio,
            first_payment = self.first_payment, 
            lower_taxband = self.lower_taxband, 
            base_yrly_salary = self.base_yrly_salary, 
            real_salary_mnthonmnth_growth = self.real_salary_mnthonmnth_growth, real_avg_salary_mnthonmnth_growth = self.real_avg_salary_mnthonmnth_growth, 
            last_repayment = self.last_repayment
        )

        with Pool(8) as p:
            for yearly_balance in tqdm(
                p.imap_unordered(run_trial, self.bootstrap_samples, chunksize=5000),
                total=len(self.bootstrap_samples),
            ):

                # Save balance paths
                yearly_balance_list.append(yearly_balance)

        self.yearly_balance_list = yearly_balance_list
                
        return yearly_balance_list

### Params

In [14]:
import requests
import json
import yfinance as yf
# %%
url = 'https://www.ons.gov.uk/economy/inflationandpriceindices/timeseries/l59c/mm23/data'


header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)
# https://stackoverflow.com/questions/43590153/http-error-403-forbidden-when-reading-html
data = json.loads(r.text)
cpih_mnthonmnth_df = pd.json_normalize(data, record_path=['months'])
#https://towardsdatascience.com/how-to-convert-json-into-a-pandas-dataframe-100b2ae1e0d8
cpih_mnthonmnth_df = cpih_mnthonmnth_df[['date', 'value']].apply(pd.Series)
cpih_mnthonmnth_df['value'] = cpih_mnthonmnth_df['value'].astype(float)
cpih_mnthonmnth_df['value'] = 1 + cpih_mnthonmnth_df['value'] / 100

#https://stackoverflow.com/questions/65612231/convert-string-month-year-to-datetime-in-pandas-dataframe

cpih_mnthonmnth_df['date'] = pd.to_datetime(cpih_mnthonmnth_df['date'], format='%Y %b')
cpih_mnthonmnth_df = cpih_mnthonmnth_df.set_index(['date'])

cpih_mnthonmnth_df.tail()

# %%
sp500 = yf.Ticker("IGUS.L")
hist = sp500.history(period="max", interval="1mo")
SP500_daily_returns = hist['Close'].to_frame()
SP500_daily_returns
# %%
sp500_cpi_df = SP500_daily_returns.pct_change()[1:]
sp500_cpi_df['Close'] = sp500_cpi_df['Close'] + 1

sp500_cpi_df = sp500_cpi_df.merge(cpih_mnthonmnth_df, left_index=True, right_index=True)

In [15]:
base_yrly_salary = 30000
real_salary_yronyr_growth = 1.02
real_avg_salary_yronyr_growth = 1.01

### Stationary Bootstrap

In [16]:
sim_st = PortfolioSim(sp500_cpi_df, real_salary_yronyr_growth,
        real_avg_salary_yronyr_growth, base_yrly_salary)

print('Generating bootstrap sample...')
sim_st.gen_bootstrap_samples(1000, kind = 'stationary', block_length = 10)

print('Running simulations...')
yearly_balance_list = sim_st.run_simulation()

  0%|          | 0/1000 [00:00<?, ?it/s]

Generating bootstrap sample...
Running simulations...


  0%|          | 0/1000 [00:01<?, ?it/s]


UnsupportedError: [1mFailed in nopython mode pipeline (step: nopython rewrites)
[1mNumba's print() function implementation does not support keyword arguments.
[1m
File "<ipython-input-13-4ac93e15ad80>", line 134:[0m
[1m<source missing, REPL/exec in use?>[0m
[0m[0m